# Алгоритм Adam

Обучим сеть LeNet на графических картах GPU. Но прежде рассмотрим ещё один алгоритм оптимизации, который поможет улучшить результаты.

Градиентный спуск (SGD) — это не самый оптимальный алгоритм обучения нейронной сети. Если величина шага слишком маленькая, сеть будет обучаться долго, а если большая — может пропустить минимум. Чтобы подбор шага был автоматическим, применяют алгоритм **Adam** _(от англ. adaptive moment estimation, «адаптивность на основе оценки моментов»)_. Он подбирает различные параметры для разных нейронов, что также ускоряет обучение модели.

Чтобы понять, как этот алгоритм работает, рассмотрим визуализацию Эмильена Дюпона из Оксфордского университета. В ней четыре алгоритма: слева SGD, справа Adam, а между ними — два похожих на Adam алгоритма (их разбирать не будем). Быстрее всех минимум находит Adam.

Запишем алгоритм Adam в Keras:

`model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])`

Чтобы настроить гиперпараметры, подключим класс алгоритма:

`from tensorflow.keras.optimizers import Adam optimizer = Adam()`

`model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])`

Основной настраиваемый гиперпараметр в алгоритме Adam — скорость обучения (learning rate). Это шаг градиентного спуска, с которого алгоритм стартует. Записывается так:

`optimizer = Adam(lr=0.01)`

По умолчанию он равен 0.001. Уменьшение шага иногда может замедлить обучение, но улучшить итоговое качество модели.

Сначала ваш код должен пройти предварительную проверку, а затем его поставят в очередь на обучение. В это время вы можете вернуться в основной курс и продолжить изучать уроки о компьютерном зрении. Когда пройдёт 2–3 часа, загляните в этот урок и 
проверьте, не завершилось ли обучение модели. Перейти к следующей задаче вы сможете только после того, как решите эту.

In [1]:
# Подключение библиотеки
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, AvgPool2D, Conv2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Загрузка обучающей выборки
def load_train(path):
    # Получение целевого признака и признака
    features_train = np.load(path + 'train_features.npy')
    target_train = np.load(path + 'train_target.npy')
    
    # Приводим значения пикселей к диапазону [0, 1]
    features_train = features_train.reshape(-1, 28, 28, 1) / 255.
    
    return features_train, target_train 

In [3]:
# Создание модели
def create_model(input_shape):
    # Инициализируем модель машинного обучения
    model = Sequential()
    
    model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', input_shape=input_shape, activation='relu'))
    
    model.add(AvgPool2D(pool_size=(2, 2), strides=None, padding='valid'))
    
    model.add(Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='relu'))
    
    model.add(AvgPool2D(pool_size=(2, 2), strides=None, padding='valid'))
    
    model.add(Flatten())

    model.add(Dense(units=120, input_shape=input_shape, activation='relu'))
    
    model.add(Dense(units=84, activation='relu'))
    
    model.add(Dense(units=10, activation='softmax'))
    
    # Для настройки гиперпараметров: Основной настраиваемый гиперпараметр в алгоритме Adam — скорость обучения (learning rate). 
    # Это шаг градиентного спуска, с которого алгоритм стартует.
    # По умолчанию он равен 0.001. Уменьшение шага иногда может замедлить обучение, но улучшить итоговое качество модели.
    optimizer_adam = Adam(lr=0.01)
    
    # Также устанавливаем параметры, отвечающие за обучение
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    
    return model

In [4]:
# Запуск модели
def train_model(model, train_data, test_data, batch_size=32, epochs=10, steps_per_epoch=None, validation_steps=None):
    # Деление обучающие выборки на признак и целевой признак
    features_train, target_train = train_data
    # Деление тестовой выборки на признак и целевой признак
    features_test, target_test = test_data
    # Обучаем модель
    model.fit(features_train, target_train, 
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model 

# Вывод на консольное окно

```
2023-05-11 21:41:15.532867: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2023-05-11 21:41:15.534503: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
2023-05-11 21:41:16.346028: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-05-11 21:41:16.355707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-11 21:41:16.355767: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-11 21:41:16.355798: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-05-11 21:41:16.357591: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-05-11 21:41:16.357960: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-05-11 21:41:16.359930: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-05-11 21:41:16.361060: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-05-11 21:41:16.361113: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-05-11 21:41:16.365331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
Using TensorFlow backend.
2023-05-11 21:41:17.001927: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-05-11 21:41:17.008937: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2099995000 Hz
2023-05-11 21:41:17.009468: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x44c4840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-05-11 21:41:17.009497: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-05-11 21:41:17.151900: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x43e5000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-11 21:41:17.151938: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-05-11 21:41:17.154289: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-11 21:41:17.154349: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-11 21:41:17.154359: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-05-11 21:41:17.154384: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-05-11 21:41:17.154395: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-05-11 21:41:17.154405: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-05-11 21:41:17.154416: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-05-11 21:41:17.154423: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-05-11 21:41:17.158730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-05-11 21:41:17.158787: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-11 21:41:17.469485: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-05-11 21:41:17.469533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2023-05-11 21:41:17.469541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2023-05-11 21:41:17.473857: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-05-11 21:41:17.473909: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10240 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
2023-05-11 21:41:18.833371: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-05-11 21:41:19.109579: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
60000/60000 - 7s - loss: 0.5671 - acc: 0.7901 - val_loss: 0.4547 - val_acc: 0.8334
Epoch 2/10
60000/60000 - 5s - loss: 0.3861 - acc: 0.8580 - val_loss: 0.3942 - val_acc: 0.8622
Epoch 3/10
60000/60000 - 5s - loss: 0.3287 - acc: 0.8802 - val_loss: 0.3354 - val_acc: 0.8791
Epoch 4/10
60000/60000 - 5s - loss: 0.2960 - acc: 0.8918 - val_loss: 0.3107 - val_acc: 0.8881
Epoch 5/10
60000/60000 - 5s - loss: 0.2714 - acc: 0.8992 - val_loss: 0.3002 - val_acc: 0.8946
Epoch 6/10
60000/60000 - 5s - loss: 0.2526 - acc: 0.9050 - val_loss: 0.2986 - val_acc: 0.8921
Epoch 7/10
60000/60000 - 5s - loss: 0.2373 - acc: 0.9114 - val_loss: 0.2790 - val_acc: 0.8992
Epoch 8/10
60000/60000 - 5s - loss: 0.2217 - acc: 0.9164 - val_loss: 0.2881 - val_acc: 0.8989
Epoch 9/10
60000/60000 - 5s - loss: 0.2100 - acc: 0.9218 - val_loss: 0.2789 - val_acc: 0.9027
Epoch 10/10
60000/60000 - 5s - loss: 0.2003 - acc: 0.9244 - val_loss: 0.2748 - val_acc: 0.9034
10000/10000 - 1s - loss: 0.2748 - acc: 0.9034
```